In [5]:
import re
from os import listdir

path = 'naver'

def ngram(s, n=2):
    rst = list()
    for i in range(len(s)-(n-1)):
        rst.append(''.join(s[i:i+n]))
    return rst

def fileids(path, ext='txt'):
    files = list(filter(lambda f:re.search(f'{ext}$', f),
                       listdir(path)))
    return list(map(lambda f:f'{path}/{f}', files))

In [6]:
from collections import Counter

gram1 = list()
gram2 = list()
gram3 = list()
gram4 = list()

for f in fileids('naver'):
    with open(f, 'r', encoding='utf8') as fp:
        doc = re.sub(r'\s+', ' ', fp.read())
        gram1.extend(ngram(doc, 1))
        gram2.extend(ngram(doc, 2))
        gram3.extend(ngram(doc, 3))
        gram4.extend(ngram(doc, 4))
        
gram1 = Counter(gram1)
gram2 = Counter(gram2)
gram3 = Counter(gram3)
gram4 = Counter(gram4)

In [7]:
len(gram1), len(gram2), len(gram3), len(gram4)

(1546, 35495, 126100, 256476)

In [8]:
'코스피가 이스라엘-팔레스타인 전쟁에 따른 국제정세 불안 속에서도 장초반 1% 넘게 오르고 있다.'
data = re.sub(r'\s', '', '코스피가 이스라엘-팔레스타인 전쟁에 따른 국제정세 불안 속에서도 장초반 1% 넘게 오르고 있다.')

In [11]:
#다음글자가 ' '인지 보는 함수
#     => 확률적으로 가장 큰 다음글자를 return
# N-gram(bigram;(A,B*)/(A*), trigram;(A,B,C*)/(A,B*))
def spacing(s):
    gram = {1:gram1, 2:gram2, 3:gram3, 4:gram4}
    given = gram[len(s)]
    candi = gram[len(s)+1]

    c = dict(filter(lambda r:re.match(s, r[0]), candi.items()))

    maxkey = ' '
    if len(c) > 0:
        maxkey = max(c, key=c.get)
        # print(c)
        # print(maxkey, c[maxkey])

    return maxkey
spacing('학교')[-1]

' '

In [13]:
N = 2
rst = ''
data = '한 번 저지른 일은 되돌릴 수 없다며 강하게 방류 중단을 요구하고 있습니다.'.replace(' ', '')
for i in range(len(data)):
    if i < N:
        if N > 1:
            key = rst[-N:] if len(rst)>1 else data[:i+1]
            if spacing(key)[-1] == ' ':
                rst += ' '
        rst += data[i]
    else:
        if spacing(rst[-N:])[-1] == ' ':
            rst += ' '
        rst += data[i]
rst

' 한번 저지른일 은되 돌릴 수 없다며 강하게 방류 중 단을 요구하고 있습니다.'

In [14]:
# 형태소+형태소+형태소 => 어절
# 고려대는, 고려대에서는, 고려대와, 고려대로, ...
# Perplexity(고려대에서는) = P(려|고)*P(대|고려)*P(에|고려대)*...
#                      => (P(고려)/P(고))*(P(고려대)/P(고려))*...
#                      => P(고려대에서는)/P(고) > P(고려대)/P(고)

In [16]:
from nltk.tokenize import word_tokenize, regexp_tokenize

word = list()
for f in fileids('naver'):
    with open(f, 'r', encoding='utf8') as fp:
        word.extend(regexp_tokenize(re.sub(r'\s+', ' ', fp.read()),
                                   r'\b\w+\b'))
word = Counter(word)        

In [17]:
#word로부터 s로 시작하는 모든 빈도의 합 return
def find(s):
    return sum(dict(filter(lambda r:r[0].startswith(s),
                          word.items())
                   ).values())

In [18]:
key = '말했다'
before = 0.0
for i in range(1, len(key)):
    p = find(key[0:i+1])/find(key[0])
    print(key[:i+1], find(key[0:i+1])/find(key[0]))
    print('자름' if before > p else '놔둠')
    before = p

말했 0.5450236966824644
놔둠
말했다 0.48578199052132703
자름


# Branching Entropy

In [25]:
def findkey(s):
    return list(
                set(
                    list(
                    map(lambda k:k[:len(s)+1],
                        list(dict(filter(lambda r:r[0].startswith(s),
                                     word.items())
                             ).keys()
                        )))
                )
           )

In [26]:
from math import log

key = '아르헨티나'
before = 0.0
for i in range(len(key)):
    total = 0.0
    for k in findkey(key[:i+1]):
        p = find(k)/find(key[:i+1])
    #     print(k, -p*log(p))
        total += -p*log(p)
    print(key[:i+1], total, '자름' if before < total else '냅둠')
    before = total

아 3.109258778586353 자름
아르 1.5498260458782016 냅둠
아르헨 0.0 냅둠
아르헨티 0.0 냅둠
아르헨티나 0.0 냅둠


In [27]:
findkey('아르헨티나')

[]

In [28]:
raw = '''
low low low low low lowest lowest newer newer
newer newer newer newer wider wider wider
'''

# 단어 = 어절
data = dict()
for k, v in Counter(raw.split()).items():
    data[' '.join(tuple(k))+' </w>'] = v
data

{'l o w </w>': 5,
 'l o w e s t </w>': 2,
 'n e w e r </w>': 6,
 'w i d e r </w>': 3}

In [30]:
def findpair(d, n=2):
    rst = dict()

    for k, v in d.items():
        # 'l o w </w>' => ['l', 'o', 'w', '</w>']
        k = k.split()
        for i in range(len(k)-(n-1)):
            key = ' '.join(k[i:i+n])
            if key in rst:
                rst[key] += v
            else:
                rst[key] = v
    return rst

In [31]:
keys = findpair(data)
best = max(keys, key=keys.get)

In [32]:
rst = dict()
for k, v in data.items():
    rst[re.sub(best, best.replace(' ',''), k)] = v

In [33]:
for _ in range(5):
    keys = findpair(data)
    best = max(keys, key=keys.get)

    rst = dict()
    for k, v in data.items():
        rst[re.sub(best, best.replace(' ', ''), k)] = v

    data = rst
    print(best, rst)

e r {'l o w </w>': 5, 'l o w e s t </w>': 2, 'n e w er </w>': 6, 'w i d er </w>': 3}
er </w> {'l o w </w>': 5, 'l o w e s t </w>': 2, 'n e w er</w>': 6, 'w i d er</w>': 3}
l o {'lo w </w>': 5, 'lo w e s t </w>': 2, 'n e w er</w>': 6, 'w i d er</w>': 3}
lo w {'low </w>': 5, 'low e s t </w>': 2, 'n e w er</w>': 6, 'w i d er</w>': 3}
n e {'low </w>': 5, 'low e s t </w>': 2, 'ne w er</w>': 6, 'w i d er</w>': 3}


In [34]:
[k.split() for k in data]

[['low', '</w>'],
 ['low', 'e', 's', 't', '</w>'],
 ['ne', 'w', 'er</w>'],
 ['w', 'i', 'd', 'er</w>']]

In [35]:
data = dict()
for k, v in word.items():
    data[' '.join(tuple(k))] = v

for _ in range(100):
    keys = findpair(data)
    best = max(keys, key=keys.get)

    rst = dict()
    for k, v in data.items():
        rst[re.sub(best, best.replace(' ', ''), k)] = v

    data = rst
    print(best.replace(' ', ''))

으로
에서
했다
니다
하는
있다
하고
20
이다
습니다
한다
00
지난
있는
이라
까지
지만
면서
서울
10
후보
한국
부터
연구
정부
대한
들이
대표
었다
하기
5일
에는
뉴스
202
기술
통령
기자
됐다
된다
대통령
적인
통해
제공
적으로
위해
것으로
개발
후보자
보다
하게
에게
기업
미국
입니다
위원
AI
이라고
시장
지원
국내
했습니다
하다
국민
밝혔
이번
시간
가능
사진
의원
겠다
인사
이어
전기
19
이용
이나
세계
하지
전자
민주
등을
사업
관련
50
말했다
았다
밝혔다
억원
진행
스트
6일
되는
장관
도록
30
우리
아이
올해
만원
사용


# Tokenizing 

In [36]:
from nltk.corpus import stopwords

In [37]:
stop = stopwords.open(
    stopwords.fileids()[stopwords.fileids().index('english')]).read()

In [38]:
from string import punctuation

s = 'To be or not to be.'
rst = []
for w in re.sub(f'[{re.escape(punctuation)}]', '', s.lower()).split():
    if w not in stop:
        rst.append(w)
    else:
        print(f'skipped:{w}')
' '.join(rst)

skipped:to
skipped:be
skipped:or
skipped:not
skipped:to
skipped:be


''